In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import datetime
from dateutil.relativedelta import relativedelta

from cartopy import crs as ccrs, feature as cfeature
import warnings
warnings.filterwarnings('ignore')

import xarray as xr
import glob, os

from eofs.xarray import Eof
from eofs.standard import Eof as sEof # If needed

sp = ccrs.SouthPolarStereo()
pc = ccrs.PlateCarree()

In [2]:
filename = '/d6/bxw2101/oras5_ohc/all_ohc_monthly.nc'
ds = xr.open_dataset(filename, decode_times=False)

date_1 = datetime.datetime.strptime("01/16/1978", "%m/%d/%Y")
def convert_times(x):
  return (date_1 + relativedelta(hours=int(x))).replace(day=1, hour=0)
convert_times_v = np.vectorize(convert_times)
ds = ds.assign_coords(time_counter=convert_times_v(ds.time_counter))
ds = ds.rename({'time_counter': 'tdim'})

In [3]:
ohc = ds.sohtc300
clim_period = ds.sel(tdim=slice('1979-01-01', '2021-12-01'))
clim_ohc = clim_period.sohtc300
ohc_mon = ohc.groupby('tdim.month')
ohc_clim = clim_ohc.groupby('tdim.month').mean(dim='tdim')
ohc_anom = ohc_mon - ohc_clim
ohc_anom = ohc_anom.drop_vars('month')

In [4]:
ohc_anom = ohc_anom.fillna(0)

In [5]:
ohc_anom
ohc = xr.Dataset(data_vars = {"ohc_anom": ohc_anom})

In [8]:
ohc.to_netcdf('/d6/bxw2101/combined_netcdf_files/ohc_anom_old_grid.nc', mode='w',format='NETCDF4')

In [21]:
solver = Eof(ohc_anom)

In [22]:
#solver.eofs()
solver.pcs() #Works?

<xarray.DataArray 'pcs' (tdim: 533, mode: 533)>
array([[-7.52040919e+10, -9.25618258e+10,  1.55729418e+10, ...,
         4.16735649e+01,  7.22900162e+01, -1.56018438e+04],
       [-8.10600530e+10, -8.85567734e+10,  9.05034547e+09, ...,
         4.17562065e+01,  7.23004684e+01, -1.56018408e+04],
       [-8.57582305e+10, -7.72961075e+10,  1.67968246e+10, ...,
         4.17010002e+01,  7.22580414e+01, -1.56018398e+04],
       ...,
       [ 1.20845935e+11, -7.76879063e+10, -1.96514243e+10, ...,
         4.16941261e+01,  7.22881699e+01, -1.56018457e+04],
       [ 1.17859279e+11, -8.07969178e+10, -2.60038738e+10, ...,
         4.17381821e+01,  7.22789078e+01, -1.56018389e+04],
       [ 1.12430170e+11, -7.75021691e+10, -2.75322655e+10, ...,
         4.17021141e+01,  7.22763290e+01, -1.56018438e+04]], dtype=float32)
Coordinates:
  * tdim     (tdim) datetime64[ns] 1978-01-01 1978-02-01 ... 2022-05-01
  * mode     (mode) int64 0 1 2 3 4 5 6 7 8 ... 525 526 527 528 529 530 531 532